In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Cấu hình
path_to_data = r'D:\Real'
img_height = 224
img_width = 224
model_save_path = r'D:\Real\model.h5'

# Hàm load ảnh
def load_images_from_folder(folder, label):
    images, labels = [], []
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    for filename in os.listdir(folder):
        if not filename.lower().endswith(valid_extensions):
            continue
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(label)
    return images, labels

# Load dữ liệu
real_images, real_labels = load_images_from_folder(os.path.join(path_to_data, 'Real'), 1)
fake_images, fake_labels = load_images_from_folder(os.path.join(path_to_data, 'Fake'), 0)

X = np.array(real_images + fake_images) / 255.0
y = np.array(real_labels + fake_labels)

# Tách dữ liệu
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Xây dựng mô hình mới
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện
history = model.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=10,
                    validation_data=(X_val, y_val))

# Lưu mô hình
model.save(model_save_path)
print(f"Mô hình đã được lưu tại {model_save_path}")

# Vẽ biểu đồ huấn luyện
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy over Epochs")
plt.show()

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2

# Cấu hình
model_path = r'D:\Real\model.h5'
image_path = r'D:\Real\Test\test5.jpg'  # Đường dẫn đến ảnh cụ thể
img_height = 224
img_width = 224

# Load và biên dịch mô hình
try:
    model = tf.keras.models.load_model(model_path)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Biên dịch để tránh cảnh báo
    print(f"Đã tải mô hình từ {model_path}")
except Exception as e:
    print(f"Lỗi khi tải mô hình: {e}")
    exit()

# Hàm nhận diện một ảnh
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Lỗi: File không tồn tại tại {image_path}")
        exit()
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Lỗi: Không thể đọc ảnh từ {image_path}. Vui lòng kiểm tra định dạng file hoặc file có bị hỏng không.")
        exit()
    
    img = cv2.resize(img, (img_width, img_height))
    img_processed = img / 255.0
    img_processed = np.expand_dims(img_processed, axis=0)

    prediction = model.predict(img_processed)
    label = "Thật" if prediction > 0.5 else "Giả"
    prob = prediction[0][0] if prediction > 0.5 else 1 - prediction[0][0]

    print(f"Ảnh: {image_path} - Dự đoán: {label} (Xác suất: {prob:.2f})")

predict_image(image_path)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2

# Cấu hình
model_path = r'D:\Real\model.h5'
img_height = 224
img_width = 224

# Load mô hình
try:
    model = tf.keras.models.load_model(model_path)
    print(f"Đã tải mô hình từ {model_path}")
except Exception as e:
    print(f"Lỗi khi tải mô hình: {e}")
    exit()

# Hàm nhận diện qua webcam
def run_camera_detection():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở camera!")
        return

    print("Đang nhận diện qua webcam. Nhấn 'q' để thoát.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Không thể đọc frame từ webcam")
            break

        img = cv2.resize(frame, (img_width, img_height))
        img_processed = img / 255.0
        img_processed = np.expand_dims(img_processed, axis=0)

        prediction = model.predict(img_processed)
        label = "That" if prediction > 0.5 else "Gia"
        prob = prediction[0][0] if prediction > 0.5 else 1 - prediction[0][0]
        color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)

        cv2.putText(frame, f"{label} ({prob:.2f})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow("Webcam - Nhận diện", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Gọi hàm nhận diện webcam
run_camera_detection()